In [7]:
import pickle
import pandas as pd

In [3]:
# Link to original dataset: https://github.com/CrowdTruth/Short-Text-Corpus-For-Humor-Detection/blob/master/datasets/humorous_oneliners.pickle
# git versions changed the Unix new lines ('\n') to DOS lines ('\r\n') so the following step is needed

from library.dos2unixfile import dos2unixfile
dos2unixfile("datasets/original_humorous_oneliners.pickle", "datasets/humorous_oneliners.pickle")

Done. Saved 10502 bytes.


In [16]:
jokes_dataL = pd.read_pickle("datasets/humorous_oneliners.pickle", compression=None)
jokes_data = pd.DataFrame(jokes_dataL, columns = ["joke"])

In [18]:
pd.set_option('display.max_colwidth', -1)
jokes_data.head(5)

,joke
0,"Insanity is hereditary, - You get it from your children."
1,"An honest politician is one who, when bought, stays bought."
2,"You can tune a piano, but you can't tuna fish."
3,A closed mouth gathers no foot.
4,What's black and white and red all over? An embarassed zebra.


In [11]:
pddata = 